In [351]:
#from IPython.display import HTML
#import io
import numpy as np
import pandas as pd

pd.set_option('max_rows', None)

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
sheets = gc.open('RYM 1001 Performances')
round1sheet = sheets.worksheet('Round1')

In [359]:
data = round1sheet.get_all_values()
data = np.array(data)
df = pd.DataFrame(data[1:], columns=data[0], index=range(1,51))
#df.tail(60).style.set_properties(**{'text-align': 'left'})

In [360]:
df = pd.DataFrame(data[1:], columns=data[0], index=range(1,51))

# Make Group column
df = pd.melt(df)
df.columns = ['Group', 'Data']
df['Group'] = df['Group'].str[6:]

# Make additional columns
df["GroupRank"] = df["Data"].str.split(" ", n = 1, expand = True)[0].str[:-1]
df["Name"] = df["Data"].str.split(" in ", n = 1, expand = True)[0].str.split(".", n = 1, expand = True)[1].str.lstrip().str.rstrip()
df['Film'] = df["Data"].str.split(" in ", n = 1, expand = True)[1].str.split("(", n = 1, expand = True)[0].str.lstrip().str.rstrip()
df['VoteDist'] = df["Data"].str.split(")", n = 1, expand = True)[1].str.split("[", n = 1, expand = True)[0].str.strip()
df['Votes'] = df['VoteDist'].str.len()

df['Name'] = df['Name'].where(df['Name'].str[-1:] != ']', df['Name'].str[:-1].str.split('[').str[1])
df['Film'] = df['Film'].where(df['Film'].str[-1:] != ']', df['Film'].str[:-1].str.split('[').str[1])

In [361]:
df.iloc[1000,0] = 20
df.iloc[1001,0] = 20
df.drop_duplicates(inplace=True)
df.drop(df.tail(1).index,inplace=True)
df.tail()

,Group,Data,GroupRank,Name,Film,VoteDist,Votes
996,20,47. Simone Signoret in Room at the Top (1959) ...,47,Simone Signoret,Room at the Top,413,3
997,20,48. Armin Mueller-Stahl in Shine (1996) 142 [7/3],48,Armin Mueller-Stahl,Shine,142,3
998,20,49. Robert Pattinson in Cosmopolis (2012) 34 [...,49,Robert Pattinson,Cosmopolis,34,2
999,20,"50. Anders Danielsen Lie in Oslo, 31. august [...",50,Anders Danielsen Lie,"Oslo, August 31st",31,2
1000,20,51. Riccardo Cucciolla in Un flic [Dirty Money...,51,Riccardo Cucciolla,Dirty Money,0,1


In [362]:
def digitsum(stringnumber):
  if stringnumber == None:
    return 0
  return sum(int(digit) for digit in stringnumber)

df['Points'] = df['VoteDist'].apply(digitsum) + 0.01*df['Votes'] + 0.001*df['VoteDist'].str.count('4')

df['Rank'] = df['Points'].rank(ascending=False,method='min').astype(int)

df['Points'] = df['Points'].round().astype(int)

df['Votes'] = df['Votes'].where(df['Points'] >= 1, 0)

# Change column order and sort
cols=['Rank','Points','Votes','Name','Film','Group','GroupRank','VoteDist']
df = df[cols].sort_values('Rank',ascending=True)

df.index=range(1,len(df['Rank'])+1)

In [363]:
# Object of type 'int64' is not JSON serializable
df = df.astype('object')
df.head(10)
#df['VoteDist']=df['VoteDist'].astype(int)

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist
1,1,60,17,Daniel Day-Lewis,There Will Be Blood,13,1,43234444343424444
2,2,55,15,Marlon Brando,The Godfather,16,1,334244434444444
3,3,55,15,Toshiro Mifune,Seven Samurai,4,1,434444434443343
4,4,55,14,Al Pacino,Dog Day Afternoon,17,1,44444434444444
5,5,54,16,Al Pacino,The Godfather Part II,3,1,3442342443432444
6,6,54,15,Jack Nicholson,One Flew Over the Cuckoo's Nest,7,1,444443443144443
7,6,54,15,Peter O'Toole,Lawrence of Arabia,14,1,443344442442444
8,8,54,15,Philip Seymour Hoffman,The Master,3,2,442334444434344
9,9,53,16,Henry Fonda,12 Angry Men,3,3,4344244243422434
10,10,53,15,Robert De Niro,Raging Bull,13,3,234444434344442


In [364]:
nominationsheet = sheets.worksheet('Nominations')
nominations = nominationsheet.get_all_values()
nominations = np.array(nominations)
ndf = pd.DataFrame(nominations[1:], columns=nominations[0])

ndf['Name'] = ndf['Name'].str.strip()
ndf['Film'] = ndf['Film'].str.strip()
ndf['Name'] = ndf['Name'].where(ndf['Name'].str[-1:] != ']', ndf['Name'].str[:-1].str.split('[').str[1])
ndf['Film'] = ndf['Film'].where(ndf['Film'].str[-1:] != ']', ndf['Film'].str[:-1].str.split('[').str[1])
ndf['ID'] = ndf['Data'].str.split(n=1,expand=True)[0].astype(int)
ndf.head(10)

,Data,Name,Film,Year,Place of Birth,Gender,Birthday,Deathday,Age,Release Date,Genres,ID
0,166 Charlie Chaplin in The Immigrant (1917),Charlie Chaplin,The Immigrant,1917,UK,2,1889-04-16,1977-12-25,28,1917-06-17,Comedy,166
1,549 Lillian Gish in Broken Blossoms or The Yel...,Lillian Gish,Broken Blossoms or The Yellow Man and the Girl,1919,USA,1,1893-10-14,1993-02-27,25,1919-05-13,"Drama, Romance",549
2,201 Conrad Veidt in The Cabinet of Dr. Caligar...,Conrad Veidt,The Cabinet of Dr. Caligari,1920,Germany,2,1893-01-22,1943-04-03,27,1920-02-26,"Drama, Horror, Thriller, Crime",201
3,"616 Max Schreck in Nosferatu, eine Symphonie d...",Max Schreck,Nosferatu: A Symphony of Horror,1922,Germany,2,1879-09-06,1936-02-20,42,1922-02-17,"Drama, Fantasy, Horror",616
4,135 Buster Keaton in Our Hospitality (1923),Buster Keaton,Our Hospitality,1923,USA,2,1895-10-04,1966-02-01,28,1923-11-03,"Comedy, Romance",135
5,344 Harold Lloyd in Safety Last (1923),Harold Lloyd,Safety Last,1923,USA,2,1893-04-20,1971-03-08,29,1923-04-01,"Comedy, Romance, Action",344
6,"136 Buster Keaton in Sherlock, Jr. (1924)",Buster Keaton,"Sherlock, Jr.",1924,USA,2,1895-10-04,1966-02-01,28,1924-04-17,"Action, Comedy, Mystery",136
7,259 Emil Jannings in Der Letzte Mann [The Last...,Emil Jannings,The Last Laugh,1924,Switzerland,2,1884-07-23,1950-01-02,40,1924-12-23,Drama,259
8,560 Lon Chaney in He Who Gets Slapped (1924),Lon Chaney,He Who Gets Slapped,1924,USA,2,1883-04-01,1973-07-12,41,1924-11-09,"Drama, Romance, Thriller",560
9,918 ZaSu Pitts in Greed (1924),ZaSu Pitts,Greed,1924,USA,1,1894-01-03,1963-06-07,30,1924-12-04,"Drama, Crime",918


In [365]:
df['ID'] = None

for i in range(len(df)):

  try:
    name = df.iloc[i,3].strip()
    film = df.iloc[i,4].strip()
    id = int(ndf[(ndf['Name'] == name) & (ndf['Film'] == film)]['Data'].str.split().iloc[0][0])
    df.iloc[i,8] = id
  except IndexError:
    print(name,film)
    continue
  except ValueError:
    print(name,film,'ValueError')
    continue

df.index = df['ID']
#df = df.drop('ID',axis=1)
df.index.name = None
df.head()

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist,ID
210,1,60,17,Daniel Day-Lewis,There Will Be Blood,13,1,43234444343424444,210
606,2,55,15,Marlon Brando,The Godfather,16,1,334244434444444,606
942,3,55,15,Toshiro Mifune,Seven Samurai,4,1,434444434443343,942
11,4,55,14,Al Pacino,Dog Day Afternoon,17,1,44444434444444,11
16,5,54,16,Al Pacino,The Godfather Part II,3,1,3442342443432444,16


In [366]:
round7sheet = sheets.worksheet('Round7Contenders')
R7df = pd.DataFrame(np.array(round7sheet.get_all_values())[1:])

R7df['Name'] = R7df[0].str.split(" in ", n = 1, expand = True)[0]
R7df['Name'] = R7df['Name'].str.split(" ",n = 1, expand = True)[1]
R7df['Name'] = R7df['Name'].mask(R7df['Name'].str.contains("\["), R7df['Name'].str.split("[", n = 1, expand = True)[1])
R7df['Name'] = R7df['Name'].mask(R7df['Name'].str.contains("\]"), R7df['Name'].str[:-1])

R7df['Film'] = R7df[0].str.split(" in ", n = 1, expand = True)[1]
R7df['Film'] = R7df['Film'].str.split("(", n = 1, expand = True)[0]
R7df['Film'] = R7df['Film'].mask(R7df['Film'].str.contains("\["), R7df['Film'].str.split("[", n = 1, expand = True)[1])
R7df['Film'] = R7df['Film'].mask(R7df['Film'].str.contains("\]"), R7df['Film'].str.split("]", n = 1, expand = True)[0])

R7df = R7df.drop([0], axis=1)
R7df['ID'] = None

for i in range(len(R7df)):

  try:
    name = R7df.iloc[i,0].strip()
    film = R7df.iloc[i,1].strip()
    id = int(ndf[(ndf['Name'] == name) & (ndf['Film'] == film)]['Data'].str.split().iloc[0][0])
    R7df.iloc[i,2] = id
  except IndexError:
    print(name,film)
    continue
  except ValueError:
    print(name,film,'ValueError')
    continue
  
R7df.tail(5)

Jean Pierre Leaud The 400 Blows
Marcello Mastroianni La Dolce Vita


,Name,Film,ID
175,Masayuki Mori,Ugetsu,982
176,Faye Wong,Chungking Express,985
177,Kinuyo Tanaka,Sansho the Bailiff,986
178,Chishu Ryu,Tokyo Story,988
179,Song Kang-ho,Parasite,998


In [367]:
df['Year'] = None
df['Place of Birth'] = None
df['Gender'] = None
df['Birthday'] = None
df['Deathday'] = None
df['Age'] = None
df['Release Date'] = None
df['Genres'] = None
df['Round 7'] = None

for i, id in enumerate(df.index):
  try:
    df.loc[id,'Year'] = ndf[ndf['ID'] == id]['Year'].iloc[0]
    df.loc[id,'Place of Birth'] = ndf[ndf['ID'] == id]['Place of Birth'].iloc[0]
    df.loc[id,'Gender'] = ndf[ndf['ID'] == id]['Gender'].iloc[0]
    df.loc[id,'Birthday'] = ndf[ndf['ID'] == id]['Birthday'].iloc[0]
    df.loc[id,'Deathday'] = ndf[ndf['ID'] == id]['Deathday'].iloc[0]
    df.loc[id,'Age'] = ndf[ndf['ID'] == id]['Age'].iloc[0]
    df.loc[id,'Release Date'] = ndf[ndf['ID'] == id]['Release Date'].iloc[0]
    df.loc[id,'Genres'] = ndf[ndf['ID'] == id]['Genres'].iloc[0]
    df.loc[id,'Round 7'] = id in R7df['ID'].unique()
  except IndexError:
    print(i)
    continue
  except ValueError:
    print('ValueError:', i, id)
    continue

df.tail()

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist,ID,Year,Place of Birth,Gender,Birthday,Deathday,Age,Release Date,Genres,Round 7
614,991,0,0,Massimo Troisi,What Time Is It?,8,50,0,614,1989,Italy,0,1953-02-19,1994-06-04,36,1989-09-27,"Comedy, Drama",False
627,991,0,0,Meryl Streep,Out of Africa,19,49,0,627,1985,USA,1,1949-06-22,None,36,1985-12-20,"History, Romance, Drama",False
182,991,0,0,Christopher Walken,The Comfort of Strangers,14,50,0,182,1990,USA,2,1943-03-31,None,46,1990-01-01,"Romance, Drama",False
726,991,0,0,Queen Latifah,Set It Off,15,50,0,726,1996,USA,1,1970-03-18,None,26,1996-11-06,"Crime, Drama, Thriller",False
737,991,0,0,Riccardo Cucciolla,Dirty Money,20,51,0,737,1972,Italy,2,1924-09-05,1999-09-17,48,1972-09-30,"Crime, Action, Thriller",False


In [ ]:
cols = ['Rank', 'Points', 'Votes', 'Name', 'Film', 'Group', 'GroupRank', 'VoteDist', 'Year', 'Place of Birth', 'Gender', 'Birthday', 'Deathday', 'Age', 'Release Date', 'Genres', 'ID', 'Round 7']
df = df[cols]

round1compiledsheet = gc.open('RYM 1001 Performances').worksheet('Round1Compiled')

def getListIndex(nrow, ncol,row_pos, col_pos):
    list_pos = row_pos*ncol + col_pos
    return(list_pos)

count_row = df.shape[0]
count_col = df.shape[1]

# note this outputs data from the 2nd row
cell_list = round1compiledsheet.range(2,1,count_row+1,count_col)

for row in range(0,count_row):
    for col in range(0,count_col):
        list_index = getListIndex(count_row, count_col, row, col)
        cell_list[list_index].value = df.iloc[row,col]

round1compiledsheet.update_cells(cell_list)

{'spreadsheetId': '1aDqnSxAtr2jMKpfs6httGwOXAe5bkQPkB0aw7wRzI3Y',
 'updatedCells': 18018,
 'updatedColumns': 18,
 'updatedRange': 'Round1Compiled!A2:R1002',
 'updatedRows': 1001}

### FINAL RESULTS

In [477]:
finalsheet = sheets.worksheet('Final')
finaldf = pd.DataFrame(np.array(finalsheet.get_all_values()))
#finaldf.tail(60).style.set_properties(**{'text-align': 'left'})

In [478]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  
    
for i in range(501):
  string = finaldf.iloc[i].values[0]
  string = list(string)
  for j, c in enumerate(string):
    if c == '.':
      break
    if c == ' ':
      string[j] = '. '
      finaldf.iloc[i].values[0] = listToString(string)
      break

#finaldf.to_csv(path_or_buf='./test.csv',index=False)
finaldf.head()

,0
0,1. Maria Falconetti as Jeanne d'Arc in La pass...
1,2. Al Pacino as Michael Corleone in The Godfat...
2,3. Giulietta Masina as Maria Ceccarelli in Le ...
3,4. Robert De Niro as Travis Bickle in Taxi Dri...
4,5. 三船敏郎 [Toshiro Mifune] as Kikuchiyo in 七人の侍 ...


In [479]:
finaldf['Rank'] = finaldf[0].str.split(" ", n = 1, expand = True)[0]#.str.split("(", n = 1, expand = True)[0].str.lstrip().str.rstrip()

# ranks 1-500
topdf = finaldf[:500]
topdf['Name'] = topdf[0].str.split(" as ", n = 1, expand = True)[0].str.split(". ", n = 1, expand = True)[1]
topdf['Film'] = topdf[0].str.split(" in ", n = 1, expand = True)[1].str[:-6]

# ranks 501-1001
bottomdf = finaldf[500:]
bottomdf['Name'] = bottomdf[0].str.split(" in ", n = 1, expand = True)[0].str.split(". ", n = 1, expand = True)[1]
bottomdf['Film'] = bottomdf[0].str.split(" in ", n = 1, expand = True)[1].str.split(" \(", n = 1, expand = True)[0]

finaldf = pd.concat([topdf,bottomdf])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [480]:
finaldf.drop(0,axis=1,inplace=True)
finaldf['Rank'] = finaldf['Rank'].str[:-1]
finaldf.tail()

,Rank,Name,Film
996,997,Queen Latifah,Set It Off
997,998,Riccardo Cucciolla,Un flic [Dirty Money]
998,999,Giacomo Rizzo,The Family Friend
999,999,Juli Jakab,Sunset
1000,999,Mariana di Girolamo,Ema


In [481]:
finaldf['Name'] = finaldf['Name'].mask(finaldf['Name'].str.contains("\["), finaldf['Name'].str.split("[", n = 1, expand = True)[1])
finaldf['Name'] = finaldf['Name'].mask(finaldf['Name'].str.contains("\]"), finaldf['Name'].str[:-1])
finaldf['Film'] = finaldf['Film'].mask(finaldf['Film'].str.contains("\["), finaldf['Film'].str.split("[", n = 1, expand = True)[1])
finaldf['Film'] = finaldf['Film'].mask(finaldf['Film'].str.contains("\]"), finaldf['Film'].str.split("]", n = 1, expand = True)[0])

finaldf.head()

,Rank,Name,Film
0,1,Maria Falconetti,The Passion of Joan of Arc
1,2,Al Pacino,The Godfather Part II
2,3,Giulietta Masina,Nights of Cabiria
3,4,Robert De Niro,Taxi Driver
4,5,Toshiro Mifune,Seven Samurai


In [482]:
finaldf['ID'] = None

for i in range(len(finaldf)):

  try:
    name = finaldf.iloc[i,1].strip()
    film = finaldf.iloc[i,2].strip()
    id = int(ndf[(ndf['Name'] == name) & (ndf['Film'] == film)]['Data'].str.split().iloc[0][0])
    finaldf.iloc[i,3] = id
  except IndexError:
    print(i,name,film)
    continue
  except ValueError:
    print(i,name,film,'ValueError')
    continue

In [483]:
finaldf.head(10)

,Rank,Name,Film,ID
0,1,Maria Falconetti,The Passion of Joan of Arc,583
1,2,Al Pacino,The Godfather Part II,16
2,3,Giulietta Masina,Nights of Cabiria,329
3,4,Robert De Niro,Taxi Driver,756
4,5,Toshiro Mifune,Seven Samurai,942
5,6,Daniel Day-Lewis,There Will Be Blood,210
6,7,Peter O'Toole,Lawrence of Arabia,714
7,8,Gloria Swanson,Sunset Blvd.,332
8,9,Gena Rowlands,A Woman Under the Influence,303
9,10,Peter Lorre,M,707


In [484]:
finaldf['Year'] = None
finaldf['Place of Birth'] = None
finaldf['Gender'] = None
finaldf['Birthday'] = None
finaldf['Deathday'] = None
finaldf['Age'] = None
finaldf['Release Date'] = None
finaldf['Genres'] = None
finaldf['R1'] = None

if 'ID' in finaldf.columns: 
  finaldf.set_index(finaldf['ID'],inplace=True)
  finaldf.drop(['ID'],inplace=True,axis=1)

for i, id in enumerate(finaldf.index):

  try:
    finaldf.loc[id,'Year'] = ndf[ndf['ID'] == id]['Year'].iloc[0]
    finaldf.loc[id,'Place of Birth'] = ndf[ndf['ID'] == id]['Place of Birth'].iloc[0]
    finaldf.loc[id,'Gender'] = ndf[ndf['ID'] == id]['Gender'].iloc[0]
    finaldf.loc[id,'Birthday'] = ndf[ndf['ID'] == id]['Birthday'].iloc[0]
    finaldf.loc[id,'Deathday'] = ndf[ndf['ID'] == id]['Deathday'].iloc[0]
    finaldf.loc[id,'Age'] = ndf[ndf['ID'] == id]['Age'].iloc[0]
    finaldf.loc[id,'Release Date'] = ndf[ndf['ID'] == id]['Release Date'].iloc[0]
    finaldf.loc[id,'Genres'] = ndf[ndf['ID'] == id]['Genres'].iloc[0]
    finaldf.loc[id,'R1'] = df[df['ID'] == id]['Rank'].iloc[0]
  except IndexError:
    print('IndexError:', i, id, finaldf.loc[id,'Name'], finaldf.loc[id,'Film'])
    continue
  except ValueError:
    print('ValueError:', i, id, finaldf.loc[id,'Name'], finaldf.loc[id,'Film'])
    continue

In [485]:
finaldf['Gender'] = finaldf['Gender'].mask(finaldf['Gender'] == '2', 'm')
finaldf['Gender'] = finaldf['Gender'].mask(finaldf['Gender'] == '1', 'f')
finaldf['Gender'] = finaldf['Gender'].mask(finaldf['Gender'] == '0', '')
finaldf['Gender'] = finaldf['Gender'].astype('category')

finaldf['Diff'] = finaldf['R1'].astype(int) - finaldf['Rank'].astype(int)
finaldf = finaldf.astype('object')
finaldf.head()

,Rank,Name,Film,Year,Place of Birth,Gender,Birthday,Deathday,Age,Release Date,Genres,R1,Diff
ID,,,,,,,,,,,,,
583,1,Maria Falconetti,The Passion of Joan of Arc,1928,France,f,1892-07-21,1946-12-12,35,1928-04-21,"Drama, History",17,16
16,2,Al Pacino,The Godfather Part II,1974,USA,m,1940-04-25,None,34,1974-12-20,"Drama, Crime",5,3
329,3,Giulietta Masina,Nights of Cabiria,1957,Italy,f,1921-02-22,1994-03-23,36,1957-10-03,Drama,78,75
756,4,Robert De Niro,Taxi Driver,1976,USA,m,1943-08-17,None,32,1976-02-08,"Crime, Drama",19,15
942,5,Toshiro Mifune,Seven Samurai,1954,China,m,1920-04-01,1997-12-24,34,1954-04-26,"Action, Drama",3,-2


In [486]:
finalcols = ['Rank', 'Diff', 'Name', 'Film', 'Year', 'Place of Birth', 'Gender', 'Birthday', 'Deathday', 'Age', 'Release Date', 'Genres']
finaldf = finaldf[finalcols]

finalcompiledsheet = gc.open('RYM 1001 Performances').worksheet('FinalCompiled')

def getListIndex(nrow, ncol,row_pos, col_pos):
    list_pos = row_pos*ncol + col_pos
    return(list_pos)

count_row = finaldf.shape[0]
count_col = finaldf.shape[1]

# note this outputs data from the 2nd row
cell_list = finalcompiledsheet.range(2,1,count_row+1,count_col)

for row in range(0,count_row):
    for col in range(0,count_col):
        list_index = getListIndex(count_row, count_col, row, col)
        cell_list[list_index].value = finaldf.iloc[row,col]

finalcompiledsheet.update_cells(cell_list)

{'spreadsheetId': '1aDqnSxAtr2jMKpfs6httGwOXAe5bkQPkB0aw7wRzI3Y',
 'updatedCells': 12012,
 'updatedColumns': 12,
 'updatedRange': 'FinalCompiled!A2:L1002',
 'updatedRows': 1001}